In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib

database_name = "TSX_Quality.sqlite"

### Utility function to get prices from our local database  

In [2]:
def get_prices_for(symbol):
    conn = sqlite3.connect(database_name)
    
    # Make sur all dates are in ascending order
    sql = f"SELECT * FROM Prices_Daily WHERE Ticker = '{symbol}' ORDER BY UPPER(Ticker) ASC, Date ASC"
    prices = pd.read_sql_query(sql, conn, index_col="Date")
    prices.index = pd.to_datetime(prices.index)
    
    # Clean up CSV data to make sure we have only floats and no "-" values
    prices.replace("-", np.NaN, inplace=True)
    #prices["Volume"].replace(0, np.NaN, inplace=True)
    prices["Open"] = prices["Open"].astype(float)
    prices["High"] = prices["High"].astype(float)
    prices["Low"]  = prices["Low"].astype(float)
    prices["Close"]= prices["Close"].astype(float)
    
    # Required for finTA
    prices.rename(columns={"Open":"open", "High":"high", "Low":"low", "Close":"close", "Volume":"volume"}, inplace=True)
    prices.index.rename("date", inplace=True) 
    
    return prices

### Indicator calculations added to the prices dataframe

In [35]:
symbol = "SHOP"
prices_df = get_prices_for(symbol)
prices_df


,Ticker,open,high,low,close,volume
date,,,,,,
2014-08-26,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-08-27,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-08-28,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-08-29,SHOP,NaN,NaN,NaN,0.040000,0.0
2014-09-01,SHOP,NaN,NaN,NaN,0.040000,0.0
...,...,...,...,...,...,...
2022-01-18,SHOP,1354.670044,1361.140015,1291.650024,1321.040039,282500.0
2022-01-19,SHOP,1334.380005,1399.000000,1287.079956,1289.819946,283600.0
2022-01-20,SHOP,1311.979980,1350.239990,1279.170044,1282.189941,159000.0


In [37]:
# Let's add all our values to calculate teh ATR indicators 
prices_df["prev_close"]        = prices_df["close"].shift()
prices_df["high-low"]          = prices_df["high"] - prices_df["low"]
prices_df["high-prev_close"]   = np.abs(prices_df["high"] - prices_df["prev_close"])
prices_df["low-prev_close"]    = np.abs(prices_df["low"] - prices_df["prev_close"])
prices_df["true_range"]        = prices_df[["high-low", "high-prev_close", "low-prev_close"]].max(axis=1)
    
prices_df["ATR7"] = prices_df["true_range"].rolling(window=7).mean()

# Supertrend indicator
# basic_upper_band = ((high + low) / 2) + (multiplier * atr)
# basic_lower_band = ((high + low) / 2) - (multiplier * atr)
multiplier = 3
prices_df["basic_upper_band"] = (prices_df["high"] + prices_df["low"]) / 2 + (multiplier * prices_df["ATR7"])
prices_df["basic_lower_band"] = (prices_df["high"] + prices_df["low"]) / 2 - (multiplier * prices_df["ATR7"])

prices_df


,Ticker,open,high,low,close,volume,prev_close,high-low,high-prev_close,low-prev_close,true_range,ATR14,ATR7,basic_upper_band,basic_lower_band
date,,,,,,,,,,,,,,,
2014-08-26,SHOP,NaN,NaN,NaN,0.040000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-27,SHOP,NaN,NaN,NaN,0.040000,0.0,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-28,SHOP,NaN,NaN,NaN,0.040000,0.0,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-08-29,SHOP,NaN,NaN,NaN,0.040000,0.0,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-01,SHOP,NaN,NaN,NaN,0.040000,0.0,0.040000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-18,SHOP,1354.670044,1361.140015,1291.650024,1321.040039,282500.0,1389.550049,69.489990,28.410034,97.900024,97.900024,83.485715,83.722848,1577.563564,1075.226475
2022-01-19,SHOP,1334.380005,1399.000000,1287.079956,1289.819946,283600.0,1321.040039,111.920044,77.959961,33.960083,111.920044,85.521432,87.708566,1606.165676,1079.914280
2022-01-20,SHOP,1311.979980,1350.239990,1279.170044,1282.189941,159000.0,1289.819946,71.069946,60.420044,10.649902,71.069946,86.516429,84.808559,1569.130694,1060.279340
